**Text to image generation with Stable Diffusion and Discord webhook integration**

*1. Click the first play button to install dependencies*

*2. Click the second play button after pasting in your Discord webhook URL*

*3. Click the third play button and the UI will appear below*

Prompt crafting ideas: https://lexica.art/

In [ ]:
!pip install transformers gradio scipy ftfy "ipywidgets>=7,<8" datasets diffusers
!pip install discord discord-webhook
!mkdir images

In [ ]:
# Paste in your Discord webhook URL (Edit Channel > Integrations > Webhooks)
WEBHOOK_URL = "https://discord.com/api/webhooks/example/webhook_URL"

In [ ]:
from gradio.components import Slider
import gradio as gr
import torch
from torch import autocast
from diffusers import StableDiffusionPipeline
import discord
from discord_webhook import DiscordWebhook, DiscordEmbed
from PIL import Image

# Uncomment the model you wish to use, original (CompVis) or anime (hakurei)
#model_id = "CompVis/stable-diffusion-v1-4"
model_id = "hakurei/waifu-diffusion"
device = "cuda"

# Diffusion pipeline object
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16, revision='fp16')
pipe = pipe.to(device)

# Gradio block
block = gr.Blocks(css=".container { max-width: 1200px; margin: auto; }")

# Enable Discord webhook
discord = True

# Main image generation function, added second parameter for num_samples for slider
def infer(prompt, num_samples, cfg_slider):
    with autocast("cuda"):
        images = pipe([prompt] * num_samples, guidance_scale=cfg_slider)["sample"]
        # Discord stuff
        webhook = DiscordWebhook(url=WEBHOOK_URL)
        i = 1
        for image in images:
            prompt = prompt.replace("'","").replace('"','')
            image.save(fp=f"images/{prompt}-{i}.png", format="PNG")
            if discord:
                embed = DiscordEmbed(title='', description=prompt, color='03b2f8')            
                with open(f"images/{prompt}-{i}.png", "rb") as f:
                    webhook.add_file(file=f.read(), filename=f"images/{prompt}-{i}.png")
                embed.set_thumbnail(url=f'attachment://images/{prompt}-{i}.png')
            i += 1
        if discord:
            webhook.add_embed(embed)
            response = webhook.execute()
    return images

# Disable NSFW check
def dummy(images, **kwargs):
    return images, False
pipe.safety_checker = dummy

# Gradio app
with block as demo:
    gr.Markdown("<h1><center>Waifu Diffusion</center></h1>")
    gr.Markdown(
        "waifu-diffusion is a latent text-to-image diffusion model that has been conditioned on high-quality anime images through fine-tuning."
    )
    with gr.Group():
        with gr.Box():
            with gr.Row().style(mobile_collapse=False, equal_height=False):

                text = gr.Textbox(
                    label="Enter your prompt", show_label=False, max_lines=1, placeholder="Example: Your idea, an art style or artist, more detail"
                ).style(
                    border=(True, False, True, True),
                    rounded=(True, False, False, True),
                    container=False
                )
                btn = gr.Button("Run").style(
                    margin=False,
                    rounded=(False, True, True, False)
                )
            # Add slider for num_samples
            with gr.Row().style(mobile_collapse=False, equal_height=False):                
                samples_slider = gr.Slider(1, 3, 2, label="Number of Samples", step=1)
            # Add slider for cfg scale
            with gr.Row().style(mobile_collapse=False, equal_height=False):                
                cfg_slider = gr.Slider(5, 15, 7.5, label="Enforce prompt accuracy", step=0.5)               
        gallery = gr.Gallery(label="Generated images", show_label=False).style(
            grid=[3], height="auto"
        )
        text.submit(infer, inputs=[text, samples_slider, cfg_slider], outputs=gallery)
        btn.click(infer, inputs=[text, samples_slider, cfg_slider], outputs=gallery)
    gr.Markdown(
        """___
   <p style='text-align: center'>
   Created by https://huggingface.co/hakurei
   <br/>
   Modified by duplonicus
   <br/>
   </p>"""
    )

demo.launch(debug=True)